## Trabajando con el contenido estático de una página

Vamos a comenzar trabajando en forma análoga a lo que hacíamos con las APIs, haciendo un simple request, nada más que ahora vamos a conseguir un código HTML en vez de datos bien formateadas en json o xml.

Al hacer un request estamos sacando un foto de la página: su contenido es completamente estático. Por estático nos referimos a que no vamos a tener habilitadas opciones como *scrollear* o bien clickear algún que otro botón. Esto último es importante porque muchas veces hay contenido que solo se puede acceder a través de dichos métodos. Trabajar con contenido dinámico lo vamos a ver aparte de esta notebook (cuando trabajemos, por ejemplo, con la librería *selenium*).

Dado que sacamos una foto de la página, recibimos un código que vamos a tener que *parsearlo*, es decir, convertirlo en un objeto manipulable en el que sea fácil navegar. Esto lo hacemos con el módulo *BeautifulSoup*.



In [1]:
# Importamos las librerías que vamos a utilizar
import requests
from bs4 import BeautifulSoup as BS
import time

Comencemos como prueba con el *homepage* del diario La Nación. Recordar que una vez que hacemos el request ya dejamos de interactuar con la página:

In [2]:
# Hacemos un request a la página de La Nación
response = requests.get("https://www.lanacion.com.ar")

# Vemos el contenido que nos devolvió
print(response.status_code)
print(response.content)

200
b'<!DOCTYPE html><html lang="es"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=0.5,maximum-scale=5.0,user-scalable=yes"/><meta name="theme-color" content="#ffffff"/><title>\xc3\x9altimas noticias de Argentina y el mundo - LA NACION</title><link rel="preload" as="image" fetchPriority="high" media="(min-width: 1280px)" href="https://www.lanacion.com.ar/resizer/v2/la-marcha-universitaria-en-la-plaza-de-YPIFHSREXRAFXIT4DR6NAIGC34.JPG?auth=52f4dea2def45fcbd4421cf40d9f52576fc237622583889ccf170945c9ea4697&amp;width=635&amp;height=635&amp;quality=70&amp;smart=true"/><link rel="preload" as="image" fetchPriority="high" media="(min-width: 768px) and (max-width: 1279px)" href="https://www.lanacion.com.ar/resizer/v2/la-marcha-universitaria-en-la-plaza-de-YPIFHSREXRAFXIT4DR6NAIGC34.JPG?auth=52f4dea2def45fcbd4421cf40d9f52576fc237622583889ccf170945c9ea4697&amp;width=488&amp;height=651&amp;quality=70&amp;smart=true"/><link rel="preloa

Listo! La información que nos dió el request es todo con lo que contamos. Ahora no queda otra que arremangarse y trabajar con esto. Para eso, *BeautifulSoup* nos da bastantes facilidades. Primero creamos la sopa:

In [10]:
# soup es un objeto manipulable creado a partir del contenido de la página y BeautifulSoup
soup = BS(response.content)

soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html lang="es"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1.0,minimum-scale=0.5,maximum-scale=5.0,user-scalable=yes" name="viewport"/><meta content="#ffffff" name="theme-color"/><title>Últimas noticias de Argentina y el mundo - LA NACION</title><link as="image" fetchpriority="high" href="https://www.lanacion.com.ar/resizer/v2/la-marcha-universitaria-en-la-plaza-de-YPIFHSREXRAFXIT4DR6NAIGC34.JPG?auth=52f4dea2def45fcbd4421cf40d9f52576fc237622583889ccf170945c9ea4697&amp;width=635&amp;height=635&amp;quality=70&amp;smart=true" media="(min-width: 1280px)" rel="preload"/><link as="image" fetchpriority="high" href="https://www.lanacion.com.ar/resizer/v2/la-marcha-universitaria-en-la-plaza-de-YPIFHSREXRAFXIT4DR6NAIGC34.JPG?auth=52f4dea2def45fcbd4421cf40d9f52576fc237622583889ccf170945c9ea4697&amp;width=488&amp;height=651&amp;quality=70&amp;smart=true" media="(min-width: 768px) and (max-width: 1279px)" rel="preload"/

Ahora todo lo que queda es encontrar los elementos que nos interese.
¿Cómo los identificamos? Es un trabajo de ida y vuelta, y prueba y error, así que a no frustrarse si no sale de entrada. Los pasos a seguir son:
- Ir a la página que queremos scrappear y con el botón derecho del mouse poner "inspeccionar elemento" sobre el sector de la página que nos interese.
- Luego podemos identificarlo por su *tag* o por algún atributo que nos permita identificarlo mejor, como por ejemplo, la clase (*class*).
- A veces puede ser más conveniente no apuntarle al elemento que nos interesa sino a un nodo superior (que lo contenga) y a partir de ahí navegar hacia dentro.

Investigando la página del diario La Nación encontramos que los recuadros correspondientes a los títulos y enlaces de las principales notas están dentro de bloques con atributo:
~~~
<... class = "com-title">
~~~
Como observamos que el tag puede ser *h1* o *h2*, vamos a tratar de no especificar el *tag* para no perdernos nada. Buscamos entonces todos los elementos que cumplan el requisito de la clase sin especificar el *tag*:

In [8]:
print(soup.find(name = 'h1').a.attrs['href'])

AttributeError: 'NoneType' object has no attribute 'attrs'

In [7]:
# Creamos una lista con los elementos cuya clase = "com-title" con cualquier tag
elements = soup.findAll(attrs = {"class": "com-title"})


Vemos en qué consiste la lista de elementos:

In [9]:
len(elements)

0

Cada elemento tiene por ejemplo enlace a la página (*href*) de la nota que va a estar contenido dentro de un *tag < a >*.
Entonces lo que hacemos es iterar sobre todos los elementos y encontrar el *tag a* y pedirle el valor del atributo *href*:

In [11]:
# Iteramos sobre los elementos
# Hacemos un try ... except ... por si alguno de los elementos no tiene enlace
# (puede ser que estemos agarrando más cosas de las que nos interesa)
urls = []
for element in elements:
  try:
    urls.append(element.find('a')['href'])
  except:
    pass

print(urls)

[]


In [12]:
print(len(urls))

0


Ya tenemos enlaces de páginas (¿son todos los que vemos cuando abrimos la página? ¿No habrá contenido dinámico que nos estamos perdiendo?).

### Scrappeo de la paǵina de un artículo

Veamos si podemos extraer contenido de las notas cuyos enlaces extraímos. La idea es la misma: ir a la página e inspeccionar los elementos que nos interese. Cuando las páginas corresponden a la misma plataforma es usual que la información se encuentre en el mismo lugar, por lo que aprendamos de una sola paǵina podemos quizás extrapolarlo a otras.

Agarremos un *url* (por ejemplo, alguno de los de arriba) y generemos la sopa:

In [13]:
# Url de interés
url = 'https://www.lanacion.com.ar/' + urls[0]
# Hacemos un request a dicha página
response = requests.get(url)
print(response.status_code)
# Creamos la sopa para manipular
soup = BS(response.content)

IndexError: list index out of range

Identificando dónde está el título de la nota (eso de vuelta, inspeccionando el elemento asociado), podemos extraer el texto:

In [14]:
# Buscamos el elemento cuya clase indentificamos como correspondiente al título de la nota
title = soup.find('h1') # Funcionó sólo con el nombre del tag, pero podríamos haber especificado los atributos de clase(attrs = {"class": "com-title --threexl"})
print(title.text)

Arrancó el acto central. Al menos 150 mil manifestantes se congregan en Plaza de Mayo por la marcha universitaria


Fecha de la nota:

In [15]:
date = soup.find(attrs = {"class": "com-date --twoxs"}).text
print(date)

AttributeError: 'NoneType' object has no attribute 'text'

O, urgando un poco más en el código HTML, podemos extraer la fecha mejor formateada (notar otra manera de buscar cosas con BS):

In [16]:
# Buscamos un tag del tipo "meta", cuyo atributo "property" tiene valor "article:published_time"
# A este tag le sacamos el valor que toma el atributo "content"
datetime = soup.find("meta", attrs = {"property": "article:published_time"})['content']
print(datetime)

TypeError: 'NoneType' object is not subscriptable

Veamos la descripción de la nota, que podemos encontrarla en:

In [ ]:
#description = soup.find(attrs = {"class": "com-subhead --bajada --m-xs"}).text
#print(description)

Por último veamos el cuerpo de la nota. Encontramos un nodo padre del tipo *section* cuya clase es *cuerpo__nota* (notar que tenemos que escribir todo exactamente como lo vemos en el código de la página. Cualquier error de tipeo va a resultar en que no nos devuelva nada):

In [17]:
# Buscamos un tag del tipo "section", cuyo atributo "class" tiene valor "cuerpo__nota"
body = soup.find("section", attrs = {"class": "cuerpo__nota"})

In [18]:
if body.find(name = 'tweet_embido'):
   body.find().decompose()

AttributeError: 'NoneType' object has no attribute 'find'

Usualmente, los párrafos están dentro de tags denominados *p*. Por lo que buscamos todos estos dentro del bloque asociado al cuerpo de la nota:

In [19]:
# Párrafos usualmente dentro de tags p
paragraphs = body.findAll('p')

print(paragraphs)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [20]:
# Iteramos para todo párrafo y vemos el texto contenido
# Podemos hacer una lista y concatenar todo con un salto de línea
texts = []
for p in paragraphs:
  texts.append(p.text)

body_text = '\n \n'.join(texts)

print(body_text)

NameError: name 'paragraphs' is not defined

### Iteración sobre varios urls

Una de las cosas interesantes del scrappeo es poder iterar sobre varias página y extraer información en forma sistemática. Extraigamos el título y la descripción para un conjunto de urls. Básicamente es hacer lo que hicimos para una nota varias veces:

In [ ]:
urls

['/politica/maximo-kirchner-sergio-massa-tiene-un-conocimiento-enormemente-superior-del-estado-al-de-martin-nid24102022/',
 '/politica/la-opcion-de-mantener-las-paso-se-vuelve-central-para-mantener-la-unidad-del-frente-de-todos-nid24102022/',
 '/el-mundo/se-allano-el-camino-y-por-primera-vez-en-la-historia-gran-bretana-tendra-un-primer-ministro-no-nid24102022/',
 '/economia/dolar/dolar-hoy-dolar-blue-hoy-a-cuanto-cotiza-este-lunes-24-de-octubre-nid24102022/',
 '/politica/maximo-kirchner-sergio-massa-tiene-un-conocimiento-enormemente-superior-del-estado-al-de-martin-nid24102022/',
 '/politica/la-escuela-sub-40-de-formacion-politica-que-funciona-de-semillero-para-javier-milei-nid24102022/',
 '/deportes/tenis/jessica-pegula-la-hija-del-magnate-que-le-gano-la-pulseada-a-donald-trump-y-a-bon-jovi-y-escribe-su-nid24102022/',
 '/politica/maximo-kirchner-conto-que-estaba-haciendo-cuando-se-entero-del-atentado-a-cristina-kirchner-y-que-le-nid24102022/',
 '/politica/maximo-kirchner-conto-que-est

Ojo, en este caso, los url que encontramos NO son aquellos que nos permiten directamente acceder al contenido de las páginas, sino que tenemos que anteponerles el dominio de la página (esta práctica es frecuente en muchas páginas)

In [ ]:
urls = ['https://www.lanacion.com.ar/' + url for url in urls]

In [ ]:
# Barremos en el listado de urls
# Le ponemos un try-except para ignorar errores
for url in urls:

  try:

    # Hacemos request al url actual
    response = requests.get(url)

    # Creamos la sopa
    soup = BS(response.content)

    # Identificamos el título
    title = soup.find(attrs = {"class": "com-title --threexl"}).text

    # Identificamos la descripción
    description = soup.find(attrs = {"class": "com-subhead --bajada --m-xs"}).text

    # Printeamos ambas y dejamos un espacio entre nota y nota
    print(title)
    print(description)
    print('\n\n')

  except:
    pass

Máximo Kirchner: “Creo que Cristina no va a ser candidata”
El diputado se expresó sobre la interna del Frente de Todos (FdT) y opinó que es “extraño” que Alberto Fernández vaya a las PASO



La opción de mantener las PASO se vuelve central para garantizar la unidad del Frente de Todos
Divide al oficialismo la posibilidad de suspender las primarias, un atajo al que algunos en el Gobierno ven como indispensable para evitar una ruptura



Se allanó el camino y por primera vez en la historia Gran Bretaña tendrá un primer ministro no blanco
Penny Mordaunt se retiró de la contienda para liderar el Partido Conservador



Dólar hoy, dólar blue hoy: a cuánto cotiza este lunes 24 de octubre
La divisa paralela abrió en los mismos índices que el último día hábil: a $291 para la venta y $287 para la compra; el dólar oficial, en cambio, cotiza a $151,50 para la compra y $159,50 para la venta



Máximo Kirchner: “Creo que Cristina no va a ser candidata”
El diputado se expresó sobre la interna del Fre

Genial, obtuvimos información de las principales noticias que están en la página principal de La Nación. Ahora bien, son todas las que vemos en la página? Pareciera que no. Pero entonces, dónde están las que faltan?

# Trabajando con el contenido dinámico de una página

Como vimos durante la clase, no todas las funcionalidades de una página son captadas a través de hacer un requests. Podemos pensar que esto es quedarse con una _foto_ de página al momento del pedido de información.

Pero al ir a la página real, nos damos cuenta que con eso no alcanza. Por ejemplo, en este caso particular, más y más notas aparecen en función de que vayamos yendo hacia abajo en la página.

Tratemos entonces de emular esto con nuestro código. Para esto, necesitaremos Selenium. En particular, necesitaremos:


*   Instalar el paquete Selenium, que nos permite interactuar con el contenido dinámico de las páginas
*   Descargar _drivers_ de los navegadores que querramos simular. Es decir, Selenium nos permite simular un navegador (chrome, firefox, y otros), generar un requests a un url e interactuar con la respuesta del servidor. Podremos hacer cliks, scrolear, encontrar tags y también sacar una captura de pantalla cuando ya hayamos hecho todo lo dinámico (buena práctica para ganar tiempo, sobre todo si nos sentimos más cómodxs con beautifullsoup).

Si bien esto es más fácil de hacer localmente, decidimos mostrar un ejemplo acá en colab por completitud. Más abajo queda el código estilo .py.



In [21]:
# Instalación de selenium, instalación del driver del navegador
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver


   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.5 MB 1.6 MB/s eta 0:00:07
    --------------------------------------- 0.2/10.5 MB 2.8 MB/s eta 0:00:04
   - -------------------------------------- 0.4/10.5 MB 3.4 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/10.5 MB 3.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/10.5 MB 3.9 MB/s eta 0:00:03
   ---- ----------------------------------- 1.1/10.5 MB 3.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/10.5 MB 3.8 MB/s eta 0:00:03
   ----- ---------------------------------- 1.5/10.5 MB 4.0 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/10.5 MB 3.9 MB/s eta 0:00:03
   ------- -------------------------------- 1.9/10.5 MB 3.9 MB/s eta 0:00:03
   ------- -------------------------------- 2.1/10.5 MB 4.1 MB/s eta 0:00:03
   -------- ------------------------------- 2.3/10.5 MB 4.0 MB/s eta 0:00:03
   ---

"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"apt" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [22]:
# Instanciamos una clase que nos permite darle el seteo al driver que usaremos
options = webdriver.ChromeOptions()
# Seteamos algunas opciones para que no muera en colab
options.add_argument("start-maximized") # https://stackoverflow.com/a/26283818/1689770
options.add_argument("enable-automation") # https://stackoverflow.com/a/43840128/1689770
options.add_argument("--headless") # only if you are ACTUALLY running headless
options.add_argument("--no-sandbox") # https://stackoverflow.com/a/50725918/1689770
options.add_argument("--disable-infobars") #https://stackoverflow.com/a/43840128/1689770
options.add_argument("--disable-dev-shm-usage") # https://stackoverflow.com/a/50725918/1689770
options.add_argument("--disable-browser-side-navigation") # https://stackoverflow.com/a/49123152/1689770
options.add_argument("--disable-gpu") # https://stackoverflow.com/questions/51959986/how-to-solve-selenium-chromedriver-timed-out-receiving-message-from-renderer-exc



In [24]:
# Instanciamos el navegador:
wd = webdriver.Chrome('chromedriver', # Acá, en nuestras compus, va el directorio + namefile del driver
                      options = options # Le pasamos las opciones
                      )
# Hacemos requests con el driver a la página de interés
url = 'https://www.lanacion.com.ar/'
wd.get(url) # Generamos el request

# Hagamos scroll unas veces, esperemos unos segundos hasta que cargue en cada scroll

for iteration in range(3):
    wd.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)

elements = wd.find_elements_by_class_name("com-title")

# Ahora guardamos los enlaces de cada elemento
hrefs = []
titles = []
for element in elements:
    try:
        href = element.find_element_by_tag_name("a").get_attribute("href")
        hrefs.append(href)
        titles.append(element.find_element_by_tag_name("a").get_attribute('title'))
    except:
        pass

wd.quit()

TypeError: WebDriver.__init__() got multiple values for argument 'options'

In [25]:
len(set(titles))

NameError: name 'titles' is not defined

In [26]:
print('\n \n'.join([f'{t}: {h}' for t, h in zip(titles, hrefs)]))

NameError: name 'titles' is not defined

# Algunas conclusiones que sacamos de aquí

*   Elemento de la lista
*   Elemento de la lista



Vimos un ejemplo concreto de cómo extraer contenido de ciertas páginas. Algunas aclaraciones:

- No suele ser lo mejor hacer esto desde un colab. Lo ideal es tener algún bot creado localmente.
- Si vamos a iterar sobre varias páginas tener presente recomendaciones tales como no hacer demasiadas requests en un período de tiempo, que podrían resultar en prohibirnos el acceso a diferentes páginas.
- Lo que vimos acá no maneja contenido dinámico: en particular, en el ejemplo que vimos la información va emergiendo a medida que scrolleamos.

El manejo de contenido dinámico es con el lenguaje javascript y podemos manejarlo con la librería *selenium*. Esto lo vamos a hacer corriendo el siguiente script por fuera del  (en spyder por ejemplo o desde la terminal *python script.py*). Con este podemos obtener muchos más enlaces y luego scrappear cada uno de ellos:

~~~
# -*- coding: utf-8 -*-
"""
El siguiente código es complemento de la notebook sobre el manejo
de contenido estático de una paǵina de la clase de web scrapping.
La idea aquí es que en muchas páginas la información se muestra en forma dinámica,
por lo que vamos a necesitar manejar cosas de javascript.
Esto lo podemos hacer con selenium,
que simula un navegador web como si fuera un usuario más.
"""
# ---------- Importación de librerías ----------- #

# Importamos algunos módulos de selenium que nos van a servir
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Importamos la libreŕía time para esperar cierto tiempo
import time

# ---------------------------------------------- #

# ------------ Creación del navegador --------- #

# Para que selenium ande hay que descargar un driver, que depende de cada
# navegador y sistema operativo, y se descarga en la página de selenium:
# https://selenium-python.readthedocs.io/installation.html#drivers

# Para especificar donde está se lo pasamos
# como argumento cuando inicializamos el navegador.
# Para linux, hay que convertirlo en ejecutable chmod +x driver
# Para mas detalles:
# https://stackoverflow.com/questions/42478591/python-selenium-chrome-webdriver

PATH_DRIVER = "..." # Cambiar aquí dónde está

# Creamos un navegador tipo Chrome.
# Podemos decirle que navegue en forma explícita o ímplícita.
# La segunda lo logramos descomentando la línea "--headless".

chrome_options = Options()
chrome_options.add_argument("--headless")

# Creación del navegador
driver = webdriver.Chrome(executable_path = PATH_DRIVER, options = chrome_options)

# -------------------------------------------- #

# --------- Visitamos la página -------------- #

# Visitamos la página y esperamos 5 segundos a que todo cargue bien
driver.get("https://www.lanacion.com.ar")
time.sleep(5)

# Acá manejamos algo dinámico:
# Dado que en La Nación el contenido aparece scrolleando vamos a hacer eso.
# Todos las líneas de código en javascript se pueden ejecutar
# con ".execute_script" e insertando el código correspondiente allí.
# Lo que vamos a hacer acá es scrollear varias veces
# hasta el final de la página y esperar un poco a que se cargue bien la página

for iteration in range(6):
	driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
	time.sleep(2)

# Una vez que dejamos scrollear identicamos todos los bloques asociados a las notas (ver notebook).
# Cómo se busca en selenium? https://selenium-python.readthedocs.io/locating-elements.html

elements = driver.find_elements_by_class_name("com-title")

# Ahora guardamos los enlaces de cada elemento
hrefs = []
for element in elements:
    try:
	href = element.find_element_by_tag_name("a").get_attribute("href")
	hrefs.append(href)
    except:
        pass

# Guardamos los enlaces en un archivo
fp = open('links.txt','w')
for href in hrefs:
	fp.write(href + '\n')
fp.close()

# Finalmente cerramos todas las sesiones de navegación que abrimos.
# Son necesarias ambas líneas (una cierra la pestaña, la otra toda el navegador)

driver.close()
driver.quit()
~~~